In [6]:
import xarray as xr
import pandas as pd
import getpass

############
# PARAMETERS
############

model = 'uninteracted_main_model_w_chn'
# select: uninteracted_main_model_w_chn, uninteracted_main_model

regions = ['SDN.4.11.49.163', 'THA.3.R3edeff05b7928bfc', 'USA.5.221', 'CAN.3.50.1276'] 

ref_temps = [-20, 0, 10, 20, 40]

############
# PATHWAYS
############

username = getpass.getuser()
db_root = '/mnt/CIL_labor/3_projection/deltabetas/spline_27_37_39'

if model == 'uninteracted_main_model_w_chn':
    
    proj_root = '/shares/gcp/outputs/labor/impacts-woodwork/uninteracted_main_model_w_chn/uninteracted_splines_w_chn_21_37_41_by_risk_empshare_noFE_YearlyAverageDay/rcp85/CCSM4/high/SSP3'
    rf = f'/home/{username}/repos/labor-code-release-2020/output/rf/uninteracted_reg_w_chn/uninteracted_reg_w_chn_full_response.csv'
    
elif model == 'uninteracted_main_model':
    
    proj_root = '/shares/gcp/outputs/labor/impacts-woodwork/test_rcc/uninteracted_splines_27_37_39_by_risk_empshare_noFE_YearlyAverageDay/rcp85/CCSM4/high/SSP3'
    rf = f'/home/{username}/repos/labor-code-release-2020/output/rf/uninteracted_reg_comlohi/uninteracted_reg_comlohi_full_response.csv'

else:
    
    print("Your model is unrecognized.")
    
######################
# GET REF TEMP VALUES
######################

rf = pd.read_csv(rf)

ref_low =  [rf.loc[rf.temp == ref_temp].yhat_low.values[0] for ref_temp in ref_temps]
ref_high = [rf.loc[rf.temp == ref_temp].yhat_high.values[0] for ref_temp in ref_temps]

############
# GET BETAS
############

high = [pd.read_csv(f'{db_root}/{model}/deltabeta_high{region}-2099.csv') for region in regions]
low = [pd.read_csv(f'{db_root}/{model}/deltabeta_low{region}-2099.csv')  for region in regions]

temp_bins = [df[['bin', 'T[2015]', 'T[diff]']] for df in high]

# subset columns and drop useless rows
subset_cols = ['bin', 'beta^F', 'beta^F*T[2099]-beta^I*T[2015]']
high = [df.loc[~df.bin.isin(['Total <20C', 'Total >20C', 'Total'])][subset_cols] for df in high]
low = [df.loc[~df.bin.isin(['Total <20C', 'Total >20C', 'Total'])][subset_cols] for df in low]

############
# GET SHARES
############
    
fulladapt = xr.open_dataset(f'{proj_root}/{model}.nc4').to_dataframe().reset_index()
incadapt = xr.open_dataset(f'{proj_root}/{model}-incadapt.nc4').to_dataframe().reset_index()

fulladapt_shares = [fulladapt.loc[(fulladapt.year == 2099) & (fulladapt.regions == region)]['clip'].iloc[0] for region in regions]
incadapt_shares = [incadapt.loc[(incadapt.year == 2099) & (incadapt.regions == region)]['clip'].iloc[0] for region in regions]

############################
# FUNCTION: calc_delta_beta
############################

def calc_delta_beta(temp_bins, high, low, f_share, h_share):
    
    # rename columns according to risk group
    high.columns = [col + '_high' for col in high.columns]
    low.columns = [col + '_low' for col in low.columns]
    
    # merge together
    df = (temp_bins
                   .merge(high,
                          left_on='bin', right_on='bin_high')
                   .merge(low,
                          left_on='bin', right_on='bin_low')
              )
    
    # add in the incadapt and fulladapt shares
    df['f_share'] = f_share
    df['h_share'] = h_share
    
    # rename our betas to their reference temperature
    df.rename(columns = {'beta^F_high' : 'beta^F_high_27C',
                         'beta^F_low' : 'beta^F_low_27C',
                        'beta^F*T[2099]-beta^I*T[2015]_high' : 'beta^F*T[2099]-beta^I*T[2015]_high_27C}',
                        'beta^F*T[2099]-beta^I*T[2015]_low' : 'beta^F*T[2099]-beta^I*T[2015]_low_27C}'},
                         inplace=True)
    
    # calculate betas for other reference temperatures
    for temp, low, high in zip(ref_temps, ref_low, ref_high):
        df[f'beta^F_high_{temp}C'] = df['beta^F_high_27C'] - high
        df[f'beta^F_low_{temp}C']  = df['beta^F_low_27C'] - low
    
    # calculating terms -> we do this for all the ref temps selected,
    # plus the basic delta beta with reference temperature 27C
    
    for temp in ref_temps + [27] :
        
        df[f'high_risk_term_{temp}C'] = df['f_share']      * df[f'beta^F_high_{temp}C'] * df['T[diff]']
        df[f'low_risk_term_{temp}C']  = (1- df['f_share']) * df[f'beta^F_low_{temp}C']  * df['T[diff]']
        df[f'extra_term_{temp}C']     = (df['f_share'] - df['h_share']) * (df[f'beta^F_high_{temp}C'] - df[f'beta^F_low_{temp}C']) * df['T[2015]']
        df[f'comb_response_{temp}C']  = df[f'high_risk_term_{temp}C'] + df[f'low_risk_term_{temp}C'] + df[f'extra_term_{temp}C']
        
        df[f'beta^F*T[2099]-beta^I*T[2015]_low_{temp}C'] = df[f'beta^F_low_{temp}C'] * df['T[diff]']
        df[f'beta^F*T[2099]-beta^I*T[2015]_high_{temp}C'] = df[f'beta^F_high_{temp}C'] * df['T[diff]']

    
    return df

############################
# FUNCTION: get_daily_minutes
############################

def get_daily_minutes(df, cols):
    
    numbers = [df[col].sum()/365 for col in cols]
    
    return numbers

############################
# GET RESULTS
############################

# raw results
results = [calc_delta_beta(a,b,c,d,e) for a,b,c,d,e in zip(temp_bins, high, low, fulladapt_shares, incadapt_shares)]

# convert certain columns to daily
daily_cols = []
for temp in ref_temps + [27] :
    daily_cols = daily_cols + [f'beta^F*T[2099]-beta^I*T[2015]_high_{temp}C',
                               f'beta^F*T[2099]-beta^I*T[2015]_low_{temp}C',
                               f'high_risk_term_{temp}C',
                               f'low_risk_term_{temp}C',
                               f'extra_term_{temp}C',
                               f'comb_response_{temp}C']

results_daily = [get_daily_minutes(df, daily_cols) for df in results]
daily_minutes = pd.DataFrame(results_daily, columns = daily_cols, index = regions)

# export this baby
daily_minutes.T.to_csv(f'/home/{username}/repos/labor-code-release-2020/output/diagnostics/ref_temp_comparison/{model}_daily_response.csv')